<a href="https://colab.research.google.com/github/ArtyomShabunin/SMOPA-25/blob/main/lesson_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://prana-system.com/files/110/rds_color_full.png" alt="tot image" width="300"  align="center"/> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src="https://mpei.ru/AboutUniverse/OficialInfo/Attributes/PublishingImages/logo1.jpg" alt="mpei image" width="200" align="center"/>
<img src="https://mpei.ru/Structure/Universe/tanpe/structure/tfhe/PublishingImages/tot.png" alt="tot image" width="100"  align="center"/>

---

# **Системы машинного обучения и предиктивной аналитики в тепловой и возобновляемой энергетике**  

# ***Практические занятия***


---

# Занятие №6
# Многоклассовая классификация методами машинного обучения
**26 марта 2025г.**

In [ ]:
import numpy as np
import pandas as pd

from sklearn import preprocessing
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_predict, StratifiedKFold
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, ConfusionMatrixDisplay
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, r2_score
import matplotlib.pyplot as plt

import ipywidgets as widgets
from IPython.display import display, clear_output

from tqdm import tqdm

## Загрузка данных

In [ ]:
import gdown
import warnings
warnings.filterwarnings('ignore')
gdown.download('https://drive.google.com/uc?id=1j54o4pHTm3HvaYTEtv_i4hOJGy5yNeZZ', verify=False)

data = pd.read_parquet("./data_modes.gzip")

In [ ]:
data.head()

## Сформируем датасет для решения задачи многоклассовой классификации

Выберем сигналы которые будем использовать. Исключим все признаки на основании которых мы на предыдущем занятии выделяли целевые режимы.

In [ ]:
use_columns = ['GTA1.DBinPU.Alzzo', 'GTA1.DBinPU.Bo', 'GTA1.DBinPU.DlPkf',
               'GTA1.DBinPU.DlPtgft', 'GTA1.DBinPU.DlPvf', 'GTA1.DBinPU.fi',
               'GTA1.DBinPU.hmGTD', 'GTA1.DBinPU.hmTG', 'GTA1.DBinPU.P1mvhTG',
               'GTA1.DBinPU.Pk', 'GTA1.DBinPU.Pmvh', 'GTA1.DBinPU.PmvhMOGTD',
               'GTA1.DBinPU.PmvhMOTG', 'GTA1.DBinPU.PmvyhMOGTD',
               'GTA1.DBinPU.PmvyhMOTG', 'GTA1.DBinPU.Prazrjag_navhode',
               'GTA1.DBinPU.Ptgpd', 'GTA1.DBinPU.Ptgvh', 'GTA1.DBinPU.Pvh',
               'GTA1.DBinPU.Pvyhlg', 'GTA1.DBinPU.Qtg', 'GTA1.DBinPU.Tk',
               'GTA1.DBinPU.Tn', 'GTA1.DBinPU.Tt', 'GTA1.DBinPU.Tvh1',
               'GTA1.DBinPU.Pzad']

X = data.loc[:,use_columns]

Сформируем отдельный целевой признак

In [ ]:
data['target'] = data[[
    'full_power_mode', 'partial_power_mode',
    'increas_power_mode', 'decreas_power_mode', 'start_up_mode',
    'shutdown_mode', 'stopped_state_mode']].idxmax(axis=1)
y = data.loc[:, ['target']]

In [ ]:
y.value_counts()

### Деление на тестовую и тренировочную выборки


In [ ]:
from sklearn.model_selection import train_test_split
# Разделяем с учетом дисбаланса классов
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

In [ ]:
y_train.value_counts()

In [ ]:
y_test.value_counts()

Видно, что датасет сильно несбалансирован.
При такой несбалансорованности модели будут плохо обучаеться на редких классах.
Алгоритм может игнорировать малочисленные классы, потому что минимизация ошибки будет достигаться за счет предсказания большинства.
Например, если модель всегда предсказывает full_power_mode, она все равно получит высокую точность.

<!-- попробуем его немного выровнять путем сокращения данных по наиболее часто встречающимся режимам -->

### Балансировка данных
**Oversampling (увеличение малых классов)**  
   - Повторение существующих редких примеров или их генерация.  
   - **SMOTE (Synthetic Minority Over-sampling Technique)** – создает новые точки малочисленных классов, используя линейные комбинации соседних точек.  

**Undersampling (уменьшение частых классов)**  
- Удаление случайных примеров из большинства классов.  

**Комбинация Oversampling + Undersampling**  
   - Часто лучше сначала **уменьшить большие**, а затем **увеличить малые классы**.  

In [ ]:
!pip install imblearn

#### Undersampling

In [ ]:
from imblearn.under_sampling import RandomUnderSampler
sampling_strategy = {
    "full_power_mode": 1000,
    "stopped_state_mode": 1000,
    "partial_power_mode": 1000
}
rus = RandomUnderSampler(sampling_strategy=sampling_strategy, random_state=42)
X_train_resampled, y_train_resampled = rus.fit_resample(X_train, y_train)

In [ ]:
y_train_resampled.value_counts()

In [ ]:
sampling_strategy = {
    "full_power_mode": 100,
    "stopped_state_mode": 100,
    "partial_power_mode": 100
}
rus = RandomUnderSampler(sampling_strategy=sampling_strategy, random_state=42)
X_test_resampled, y_test_resampled = rus.fit_resample(X_test, y_test)

In [ ]:
y_test_resampled.value_counts()

#### Oversampling

In [ ]:
from imblearn.over_sampling import SMOTE
X_train_resampled, y_train_resampled = SMOTE().fit_resample(X_train_resampled, y_train_resampled)

In [ ]:
y_train_resampled.value_counts()

### Нормализация или стандартизация данных

In [ ]:
# scaler = preprocessing.MinMaxScaler() # нормализация данных
scaler = preprocessing.StandardScaler() # стандартизация данных

X_train_resampled_scaled = pd.DataFrame(
    scaler.fit_transform(X_train_resampled),
    columns=X_train_resampled.columns,
    index=X_train_resampled.index)

X_test_resampled_scaled = pd.DataFrame(
    scaler.transform(X_test_resampled),
    columns=X_test_resampled.columns,
    index=X_test_resampled.index)

X_train_resampled_scaled.describe()

Инициализируем переменные для дальнейшего сравнения моделей

In [ ]:
accuracy_classifier = {}
precision_classifier = {}
recall_classifier = {}
f1_classifier = {}

Метод опорных векторов изначально предназначен для бинарной классификации, однако существуют способы адаптации его для решения задач многоклассовой классификации.  

Один из таких подходов — стратегия **"один против всех"** (*one-versus-all*, OvA), также известная как **"один против остальных"** (*one-versus-the-rest*). В этом случае обучается $N$ бинарных классификаторов, по одному для каждого класса. Во время предсказания выбирается класс с наивысшим значением функции принятия решения.  

Другой подход — **"один против одного"** (*one-versus-one*, OvO), при котором для каждой пары классов обучается отдельный бинарный классификатор. Таким образом, для $N$ классов требуется обучить $\frac{N(N - 1)}{2}$ классификаторов. Преимущество этого метода в том, что каждый классификатор тренируется только на подмножестве данных, относящихся к двум соответствующим классам.  

Метод опорных векторов плохо масштабируется при увеличении объема обучающего набора, поэтому стратегия **OvO часто предпочтительнее**: проще и быстрее обучить множество небольших классификаторов, чем несколько моделей, использующих полный набор данных. Однако для большинства бинарных алгоритмов стратегия **OvA является более естественной**, поскольку в ней каждая модель учитывает все данные и их принадлежность к конкретному классу.  

### Рекомендации по выбору стратегии:
- **OvO**  
  - Лучше для моделей, плохо масштабируемых с ростом данных (например, SVM с нелинейными ядрами).  
  - Эффективна при **большом количестве классов**, так как каждый классификатор использует только малую часть данных.  
  - Может быть избыточна при небольшом числе классов.  

- **OvR**   
  - Применяется, когда **классов немного**, чтобы избежать чрезмерного количества моделей.  
  - Может работать лучше, если один класс является **доминирующим** (например, задачи выявления аномалий).  

Выбор стратегии зависит от конкретных данных и требований к модели. Если данных много и используется метод SVM, чаще всего предпочтительнее OvO. Если же данных не так много или алгоритм легко масштабируется, стоит попробовать OvA.

In [ ]:
kernel = widgets.Dropdown(
    options=['linear', 'poly', 'rbf', 'sigmoid'],
    value='rbf',
    description='Kernel:',
    disabled=False,
)

strategy = widgets.Dropdown(
    options=['ovo', 'ovr'],
    value='ovr',
    description='Strategy:',
    disabled=False,
)

widgets.VBox([
    widgets.Label('Тип ядра, который будет использоваться в алгоритме'),
    kernel,
    widgets.Label('Стратегия'),
    strategy])

In [ ]:
svm_clf = SVC(gamma="auto", random_state=42,
              decision_function_shape=strategy.value,
              kernel = kernel.value)
svm_clf.fit(X_train_resampled_scaled, y_train_resampled)

In [ ]:
some_indexes = y_test_resampled.groupby('target').sample(n=1, random_state=42).index
X_some_modes = X_test_resampled_scaled.loc[some_indexes]
y_some_modes = y_test_resampled.loc[some_indexes]

for target, predict in zip(y_some_modes.values, svm_clf.predict(X_some_modes)):
    print(f"Истина - {target[0]} >>> {predict} - предсказание")

Метод **`decision_function()`** в `SVC` из `sklearn` возвращает оценку степени принадлежности объекта к определенному классу. Эта оценка называется **отступом (margin)** и представляет собой расстояние до разделяющей гиперплоскости в пространстве признаков.  

**Для многоклассовой классификации** (`n_classes > 2`):  
   - Если используется стратегия **"Один-против-одного" (OvO, по умолчанию в `SVC`)**, то метод возвращает массив размерности `(n_samples, n_classes * (n_classes - 1) / 2)`, где каждая колонка соответствует предсказанному отступу для одной из пар классов. Итоговый класс определяется голосованием.  
   - Если используется стратегия **"Один-против-всех" (OvR, при использовании `OneVsRestClassifier`)**, то метод возвращает массив размерности `(n_samples, n_classes)`, где каждое значение — это отступ для соответствующего класса.

In [ ]:
some_mode_scores = svm_clf.decision_function(X_some_modes)
print(some_mode_scores)
print(y_some_modes)

In [ ]:
svm_clf.classes_

### Анализ качества модели опорных векторов

In [ ]:
y_test_pred_svm = svm_clf.predict(X_test_resampled_scaled)

#### Матрица неточностей

In [ ]:
conf_mat = confusion_matrix(y_test_resampled, y_test_pred_svm)
ConfusionMatrixDisplay(conf_mat, display_labels=svm_clf.classes_).plot()
plt.xticks(rotation=90)
plt.show()

#### Accuracy

In [ ]:
accuracy_classifier['svm'] = accuracy_score(y_test_resampled, y_test_pred_svm)

#### Precision и recall

In [ ]:
precision_classifier['svm'] = precision_score(y_test_resampled, y_test_pred_svm, average='macro', zero_division = np.nan)
recall_classifier['svm'] = recall_score(y_test_resampled, y_test_pred_svm, average='macro', zero_division = np.nan)

#### F1

In [ ]:
f1_classifier['svm'] = f1_score(y_test_resampled, y_test_pred_svm, average='macro', zero_division = np.nan)

#### Значения метрик

In [ ]:
print(f"accuracy - {accuracy_classifier['svm']*100:0.2f}%")
print(f"precision - {precision_classifier['svm']*100:0.2f}%")
print(f"recall - {recall_classifier['svm']*100:0.2f}%")
print(f"F1 - {f1_classifier['svm']*100:0.2f}%")

## Метод ближайших соседей (K-Nearest Neighbors, KNN)


**Метод k-ближайших соседей (KNN)** – это простой и эффективный алгоритм **классификации** и **регрессии**, основанный на **поиске ближайших точек** в пространстве признаков.  

**Основная идея:**  
1. Для нового объекта **ищутся K ближайших точек** (соседей) в обучающей выборке.  
2. Класс объекта определяется **по большинству** среди соседей.   

![knn image](https://vitalflux.com/wp-content/uploads/2020/09/Screenshot-2020-09-22-at-2.34.57-PM.png)

---

**Как работает KNN?**  

**Выбираем количество соседей (K)**  
- Маленькое K → модель **чувствительна** к шуму (может переобучаться).  
- Большое K → модель становится **более устойчивой**, но может терять точность.  

**Вычисляем расстояния между точками**  
- **Евклидово расстояние** (стандартный вариант):  
$$ d(A, B) = \sqrt{(x_1 - x_2)^2 + (y_1 - y_2)^2} $$
- Можно использовать и другие метрики: **Манхэттенское, Косинусное, Чебышёва**.  

**Определяем класс нового объекта**  
- Берем K ближайших точек и **голосуем за класс** (большинство решает).  
- Если **регрессия** – усредняем значения.  

---

**Преимущества и недостатки KNN**  

**Плюсы:**  
**Простота** – не требует сложного обучения.  
**Гибкость** – можно менять метрики и K.  
**Хорошо работает при малых данных**.  

**Минусы:**  
**Медленный при больших данных** – O(N) сложность поиска соседей.  
**Чувствителен к шуму** при малых K.  
**Плохо работает с высокой размерностью**.  

---

Метод ближайших соседей **хорош для небольших данных** и **прост в реализации**, но **плохо масштабируется**.  
Лучше работает при **малом количестве признаков** и **нормализованных данных**.

In [ ]:
n_neighbors = widgets.IntSlider(
    value=5, min=1, max=20, step=1, description="Число соседей:")
display(n_neighbors)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn_clf = KNeighborsClassifier(n_neighbors=n_neighbors.value)
knn_clf.fit(X_train_resampled_scaled, y_train_resampled)

In [ ]:
some_indexes = y_test_resampled.groupby('target').sample(n=1, random_state=42).index
X_some_modes = X_test_resampled_scaled.loc[some_indexes]
y_some_modes = y_test_resampled.loc[some_indexes]

for target, predict in zip(y_some_modes.values, knn_clf.predict(X_some_modes)):
    print(f"Истина - {target[0]} >>> {predict} - предсказание")

Метод **`predict_proba()`** в `KNeighborsClassifier` предоставляет **вероятности** принадлежности объекта к каждому классу. Эти вероятности можно интерпретировать как **"уверенность"** модели.

Метод `predict_proba()` в **KNN** может давать высокие вероятности, даже если предсказание неверное, особенно если количество соседей мало или данные несбалансированы. Рекомендуется проводить настройку параметра `k`, балансировать данные и, в случае необходимости, рассматривать другие методы классификации.

In [ ]:
pd.DataFrame(knn_clf.predict_proba(X_some_modes), columns=svm_clf.classes_)

Для автоматического подбора гиперпараметров для модели **K ближайших соседей (KNN)** в Python можно использовать **GridSearchCV** или **RandomizedSearchCV** из библиотеки **scikit-learn**. Эти инструменты помогут подобрать оптимальные значения гиперпараметров, таких как количество соседей (k), метрика расстояния, алгоритм поиска ближайших соседей и другие параметры.

1. **Использование `GridSearchCV`**:
   - `GridSearchCV` выполняет **поиск по сетке**, проверяя все возможные комбинации заданных гиперпараметров. Это даёт гарантию, что вы найдете наилучшие параметры, но может занять много времени для больших наборов данных и большого количества параметров.

2. **Использование `RandomizedSearchCV`**:
   - `RandomizedSearchCV` выполняет **случайный поиск** по заданному пространству гиперпараметров, что может быть быстрее, чем полный перебор, особенно если пространство поиска большое.

### Ключевые параметры:
- **`n_neighbors`**: количество ближайших соседей, которое будет использовать модель для классификации.
- **`weights`**: функция веса, которая влияет на то, как рассчитываются "веса" соседей. Может быть:
  - `'uniform'`: все соседи имеют одинаковый вес.
  - `'distance'`: соседи взвешиваются в зависимости от расстояния.
- **`metric`**: метрика расстояния, например:
  - `'euclidean'` — евклидово расстояние.
  - `'manhattan'` — манхэттенское расстояние.
  - `'chebyshev'` — расстояние Чебышева.
- **`cv`**: количество фолдов для кросс-валидации.
- **`n_jobs`**: количество ядер процессора, которое будет использоваться для параллельных вычислений (например, `-1` — для использования всех доступных ядер).

#### GridSearchCV

In [ ]:
from sklearn.model_selection import GridSearchCV

# Создание модели KNN
knn = KNeighborsClassifier()

# Определение параметров для поиска
param_grid = {
    'n_neighbors': [3, 5, 7, 9, 11],  # Количество соседей
    'weights': ['uniform', 'distance'],  # Весовые функции
    'metric': ['euclidean', 'manhattan', 'chebyshev'],  # Метрики расстояния
}

# Использование GridSearchCV для подбора параметров
grid_search = GridSearchCV(estimator=knn, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)

# Обучение модели с GridSearchCV
grid_search.fit(X_train_resampled_scaled, y_train_resampled)

# Вывод лучших параметров
print("Лучшие параметры:", grid_search.best_params_)

# Оценка модели на тестовых данных
best_knn = grid_search.best_estimator_
print("Точность на тестовой выборке:", best_knn.score(X_test_resampled_scaled, y_test_resampled))

#### RandomizedSearchCV

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

# Создание модели KNN
knn = KNeighborsClassifier()

# Определение параметров для случайного поиска
param_dist = {
    'n_neighbors': np.arange(1, 21),  # Количество соседей
    'weights': ['uniform', 'distance'],  # Весовые функции
    'metric': ['euclidean', 'manhattan', 'chebyshev'],  # Метрики расстояния
}

# Использование RandomizedSearchCV для подбора параметров
random_search = RandomizedSearchCV(estimator=knn, param_distributions=param_dist, n_iter=10, cv=5, n_jobs=-1, verbose=2, random_state=42)

# Обучение модели с RandomizedSearchCV
random_search.fit(X_train_resampled_scaled, y_train_resampled)

# Вывод лучших параметров
print("Лучшие параметры:", random_search.best_params_)

# Оценка модели на тестовых данных
best_knn = random_search.best_estimator_
print("Точность на тестовой выборке:", best_knn.score(X_test_resampled_scaled, y_test_resampled))

Выбор **n_neighbors = 1** может быть оптимален, если данные достаточно чисты и между классами есть чёткое разделение. В таком случае ближайший сосед может быть достаточно хорош для точного предсказания, и использование одного соседа может быть достаточно эффективным.

### Анализ качества модели ближайших соседей

In [ ]:
y_test_pred_knn = best_knn.predict(X_test_resampled_scaled)

In [ ]:
conf_mat = confusion_matrix(y_test_resampled, y_test_pred_knn)
ConfusionMatrixDisplay(conf_mat, display_labels=svm_clf.classes_).plot()
plt.xticks(rotation=90)
plt.show()

#### Accuracy

In [ ]:
accuracy_classifier['knn'] = accuracy_score(y_test_resampled, y_test_pred_knn)

#### Precision и recall

In [ ]:
precision_classifier['knn'] = precision_score(y_test_resampled, y_test_pred_knn, average='macro', zero_division = np.nan)
recall_classifier['knn'] = recall_score(y_test_resampled, y_test_pred_knn, average='macro', zero_division = np.nan)

#### F1

In [ ]:
f1_classifier['knn'] = f1_score(y_test_resampled, y_test_pred_knn, average='macro', zero_division = np.nan)

#### Значения метрик

In [ ]:
print(f"accuracy - {accuracy_classifier['knn']*100:0.2f}%")
print(f"precision - {precision_classifier['knn']*100:0.2f}%")
print(f"recall - {recall_classifier['knn']*100:0.2f}%")
print(f"F1 - {f1_classifier['knn']*100:0.2f}%")

## Случайный лес (Random Forest)

**Случайный лес (Random Forest)** – это ансамблевый алгоритм машинного обучения, который строит **множество деревьев решений** и объединяет их предсказания. Он используется как для **классификации**, так и для **регрессии**.  

**Основная идея** – создать несколько деревьев решений и усреднять их предсказания, чтобы модель была **более точной и устойчивой**.

<!-- ![rf image](https://victorzhou.com/media/random-forest-post/random-forest.png) -->

<img src="https://victorzhou.com/media/random-forest-post/random-forest.png" alt="rf image" width="800"  align="center"/>

---

**Как работает случайный лес?**  
**Создание множества деревьев решений**  
- Каждое дерево строится на **случайной подвыборке** данных (bagging).  
- Для каждого разбиения в узле дерева выбирается **случайное подмножество признаков**.  

**Обучение деревьев независимо друг от друга**  
- В отличие от градиентного бустинга, деревья **не зависят друг от друга**.  
- Это делает алгоритм **быстрым в обучении** и **устойчивым к переобучению**.  

**Объединение предсказаний**  
- В **классификации** – голосование большинства (majority voting).  

---

**Когда использовать Random Forest?**  
Когда нужна **высокая точность**.  
Когда данные **шумные** или **содержат выбросы**.  
Когда важна **устойчивость модели** к переобучению.  
Когда нужно **оценить важность признаков**.  

**Когда НЕ использовать?**  
Если нужны **интерпретируемые модели** – сложно понять, как именно модель принимает решение.  
Если важна **скорость предсказаний** – например, для реального времени.  

---

**Random Forest** – мощный алгоритм, который отлично работает на большинстве задач машинного обучения. Он особенно полезен, когда нужна **высокая точность**, а данные **сложные и шумные**.

In [ ]:
n_estimators = widgets.IntSlider(
    value=100, min=1, max=500, step=1, description="Число деревьев:")
display(n_estimators)

In [ ]:
forest_clf = RandomForestClassifier(n_estimators=n_estimators.value, random_state=42)
forest_clf.fit(X_train_resampled_scaled, y_train_resampled)

In [ ]:
some_indexes = y_test_resampled.groupby('target').sample(n=1, random_state=42).index
X_some_modes = X_test_resampled_scaled.loc[some_indexes]
y_some_modes = y_test_resampled.loc[some_indexes]

for target, predict in zip(y_some_modes.values, forest_clf.predict(X_some_modes)):
    print(f"Истина - {target[0]} >>> {predict} - предсказание")

Метод **`predict_proba()`** в Random Forest предоставляет вероятности для каждого класса. Эти вероятности можно интерпретировать как уверенность модели в предсказаниях. Чем выше вероятность для предсказанного класса, тем более уверена модель в своём решении.

In [ ]:
forest_clf.predict_proba(X_some_modes)

#### Подбор гиперпараметров с использованием `RandomizedSearchCV`

In [ ]:
# Создание модели Random Forest
rf = RandomForestClassifier(random_state=42)

# Использование RandomizedSearchCV для подбора параметров
random_search = RandomizedSearchCV(estimator=knn, param_distributions=param_dist, n_iter=10, cv=5, n_jobs=-1, verbose=0, random_state=42)

# Определение диапазонов параметров для случайного поиска
param_dist = {
    'n_estimators': np.arange(50, 500, 50),  # Количество деревьев
    # 'max_depth': [None, 10, 20, 30, 40],  # Максимальная глубина деревьев
    # 'min_samples_split': np.arange(2, 20, 2),  # Минимальное количество образцов для разделения узла
    # 'min_samples_leaf': np.arange(1, 20, 2),  # Минимальное количество образцов в листьях
    # 'max_features': ['auto', 'sqrt', 'log2'],  # Количество признаков для разбиения
    # 'bootstrap': [True, False],  # Использование бутстрапа
}

# Создание RandomizedSearchCV
random_search = RandomizedSearchCV(estimator=rf, param_distributions=param_dist, n_iter=10, cv=5, n_jobs=-1, verbose=2, random_state=42)

# Обучение модели с RandomizedSearchCV
random_search.fit(X_train_resampled_scaled, y_train_resampled)

# Вывод лучших параметров
print("Лучшие параметры:", random_search.best_params_)

# Оценка модели на тестовых данных
best_rf = random_search.best_estimator_
print("Точность на тестовой выборке:", best_rf.score(X_test_resampled_scaled, y_test_resampled))

### Анализ качества модели случайного леса

In [ ]:
y_test_pred_forest = best_rf.predict(X_test_resampled_scaled)

In [ ]:
conf_mat = confusion_matrix(y_test_resampled, y_test_pred_forest)
ConfusionMatrixDisplay(conf_mat, display_labels=svm_clf.classes_).plot()
plt.xticks(rotation=90)
plt.show()

#### Accuracy

In [ ]:
accuracy_classifier['forest'] = accuracy_score(y_test_resampled, y_test_pred_forest)

#### Precision и recall

In [ ]:
precision_classifier['forest'] = precision_score(y_test_resampled, y_test_pred_forest, average='macro', zero_division = np.nan)
recall_classifier['forest'] = recall_score(y_test_resampled, y_test_pred_forest, average='macro', zero_division = np.nan)

#### F1

In [ ]:
f1_classifier['forest'] = f1_score(y_test_resampled, y_test_pred_forest, average='macro', zero_division = np.nan)

#### Значения метрик

In [ ]:
print(f"accuracy - {accuracy_classifier['forest']*100:0.2f}%")
print(f"precision - {precision_classifier['forest']*100:0.2f}%")
print(f"recall - {recall_classifier['forest']*100:0.2f}%")
print(f"F1 - {f1_classifier['forest']*100:0.2f}%")

## Сравнение

In [ ]:
df = pd.DataFrame(
    [precision_classifier, recall_classifier, f1_classifier, accuracy_classifier],
    index=['Precision', 'Recall', 'F1-score', 'Accuracy'])

In [ ]:
df

#### **Random Forest (лес случайных деревьев) – ЛУЧШАЯ модель**  
**Лучшие показатели по всем метрикам** → Precision, Recall, F1-score, Accuracy.  
**Высокий Precision и Recall** говорят о том, что модель не только правильно классифицирует объекты, но и редко ошибается.  
**Вывод**: Эта модель демонстрирует **наилучшие результаты**, что делает её оптимальным выбором.  
